id 부여

In [ ]:
import os
import pandas as pd
import mysql.connector

#DB 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='premo',
)
cursor = conn.cursor()

#team_name 추출
team_set = set()

data_root = './datas'

for folder in os.listdir(data_root):
    folder_path = os.path.join(data_root, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.csv') and 'teams' in file.lower():
                file_path = os.path.join(folder_path, file)
                try:
                    df = pd.read_csv(file_path)
                    
                    if 'common_name' in df.columns:
                        team_set.update(df['common_name'].dropna().unique())
                except Exception as e:
                    print(f"X Error in {file_path}: {e}")
                    
#db에 삽입(중복 방지)
for team in team_set:
    cursor.execute("insert into teams (team_name) values (%s)", (team,))
    
#완료
conn.commit()
cursor.close()
conn.close()

IntegrityError: 1062 (23000): Duplicate entry 'Everton' for key 'teams.team_name'

컬럼 삭제

In [ ]:
#matches
matches_drop = ['timestamp', 'date_GMT', 'status', 'attendance', 'referee', 'stadium_name']

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'matches' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                df = df.drop(columns=[col for col in matches_drop if col in df.columns])
                df.to_csv(file_path, index=False)
                print(f"[완료] 컬럼 삭제: {file_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")
                
#players
players_drop = ['birhthday', 'birthday_GMT', 'league', 'season', 'nationality']

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'palyers' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                #average_rating_overall 이후 컬럼 제거
                if 'average_rating_overall' in df.columns:
                    cut_idx = df.columns.get_loc('average_rating_overall')
                    df = df.iloc[:, :cut_idx+1]
                    
                #player_drop 컬럼 제거
                df = df.drop(columns=[col for col in players_drop if col in df.columns])
                
                #저장
                df.to_csv(file_path, index=False)
                print(f"[완료] 컬럼 삭제: {file_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")

#teams

#teams+teams2 병합
for root, dirs, files in os.walk(data_root):
    files_lower = [f.lower() for f in files]
    
    #teams와 teams2 파일이 모두 존재하는 폴더만 처리
    if any('teams' in f for f in files_lower) and any('teams2' in f for f in files_lower):
        try:
            #실제 파일명 찾기
            teams1_file = next(f for f in files if 'teams' in f.lower())
            teams2_file = next(f for f in files if 'teams2' in f.lower())
            
            #파일 경로
            path1 = os.path.join(root, teams1_file)
            path2 = os.path.join(root, teams2_file)
            
            #파일 불러오기
            teams1 = pd.read_csv(path1)
            teams2 = pd.read_csv(path2)
            
            #병합
            teams = pd.merge(teams, teams2, on='team_name', suffixes=('_t1', '_t2'))
            
            #중복 컬럼 제거
            cols_to_drop = []
            for col in teams.columns:
                if col.endswith('_t1'):
                    base = col[:-3]
                    t2_col = base + '_t2'
                    if t2_col in teams.columns and teams[col].equals(teams[t2_col]):
                        cols_to_drop.append(t2_col)
                        
            teams_clean = teams.drop(columns=cols_to_drop)
            teams_clean.columns = [c.replace('_t1','').replace('_t2','') for c in teams_clean.columns]
            
            #저장 경로
            merged_filename = teams1_file.replace('teams', 'merged_teams')
            save_path = os.path.join(root, merged_filename)
            
            teams_clean.to_csv(save_path, index=False)
            print(f"[완료] 병합 및 저장: {save_path}")
        except Exception as e:
            print(f"[오류] {root}: {e}")
       
#삭제     
teams_drop = ['team_name', 'season', 'country']
    
for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'merged_teams' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                df = df.drop(columns=[col for col in teams_drop if col in df.columns])
                
                df.to_csv(file_path, index=False)
                print(f"[완료] 컬럼 삭제: {file_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")


1. matches - home_goal_timings, away_goal_timings one-hot encoding 처리 후 매핑
2. csv 데이터 id 매핑

In [ ]:
import numpy as np

#45'2 -> 45
def parse_goal_timings(timing_str):
    if pd.isna(timing_str) or timing_str == '':
        return []